### Our Hamiltonian (+ decay)

$$H = \sum_{\langle ij \rangle} (J^x \sigma_i^x \sigma_j^x + J^y \sigma_i^y \sigma_j^y + J^z \sigma_i^z \sigma_j^z)$$

In [ ]:
from pycu_spins import CudaSpins, np
import matplotlib.pyplot as plt

#### Call the interface class with parameters for GPU and physical simulation

In [ ]:
cs = CudaSpins(
    params = 
            {
            "num_bodies": 256,
            "dt": 1e-3, 
            "tsteps": 2000,
            "save_step":1,
            "gamma": 1.0,
            "jx": 0.9,
            "jy": 0.9,
            "jz": 1.0,
            "save_start":0
            },
    gpu_params = {
        "block_size":128, 
        "use_host_mem": False}
            )

#### Check platforms on which you can simulate you system

In [ ]:
cs.get_platform_options()

#### You can select your platform GPU / CPU

In [ ]:
cs.set_platform("CPU")

#### Run simulation

In [ ]:
cs.run()

#### Process the results 

In [ ]:
tsteps = cs.params["tsteps"] // cs.params["save_step"]

time = np.linspace(0, cs.params['tsteps'] * cs.params["dt"], tsteps)
time_body = np.linspace(0, cs.params['tsteps'] * cs.params["dt"], 
tsteps*cs.params["num_bodies"])

sx = np.sqrt(3)*np.sin(cs.results[::2]) * np.sin(cs.results[1::2])
sy = -np.sqrt(3)*np.sin(cs.results[::2]) * np.cos(cs.results[1::2])
sz = -np.sqrt(3)*np.cos(cs.results[::2])

sx_mean = np.reshape(sx, (-1, cs.params["num_bodies"])).mean(axis=-1)
sy_mean = np.reshape(sy, (-1, cs.params["num_bodies"])).mean(axis=-1)
sz_mean = np.reshape(sz, (-1, cs.params["num_bodies"])).mean(axis=-1)

#### Plot

In [ ]:
plt.figure(figsize=(7,3))
plt.subplot(121)
plt.title("Intital conf")
plt.plot(time_body, sx, label="sx")
plt.plot(time_body, sy, label="sy")
plt.plot(time_body, sz, label="sz")
plt.xlabel("t x N", fontsize=14)
plt.ylabel("s", fontsize=14)
plt.subplot(122)
plt.title("Averaged conf")
plt.plot(time, sx_mean, label="<sx>")
plt.plot(time, sy_mean, label="<sy>")
plt.plot(time, sz_mean, label="<sz>")    
plt.xlabel("t", fontsize=14)
plt.ylabel("<s>", fontsize=14)
plt.legend(frameon=False, prop={"size": 10})
plt.grid(True)
plt.tight_layout()

In [ ]:
del cs

In [ ]:
system_params = {
                "num_bodies": 256,
                "dt": 1e-3,
                "tsteps": 200, #20000 
                "save_step":1, #10
                "gamma": 1.0,
                "jx": 0.9,
                "jy": 0.9,
                "jz": 1.0,
                "save_start":0
                }

gpu_settings = {
            "block_size":4,
            "use_host_mem": True}    


In [ ]:
perf_gpu = []
# for i in [12,13,14, 15, 16, 17]:

for i in [128, 2048]:
    system_params["num_bodies"] = i
    cs = CudaSpins(params=system_params, gpu_params=gpu_settings)
    cs.set_platform("GPU")
    cs.run()
    perf_gpu.append(cs.platf_time)
    del cs
    
np.save(f'perf_gpu_{gpu_settings["block_size"] }',perf_gpu)

In [ ]:
np.load('perf_cpu.npy')

In [ ]:
np.save('perf_gpu',perf_gpu)

In [1]:
import sympy as sym
from sympy.matrices import Matrix

In [64]:
theta = sym.Symbol('theta_j', real=True)
phi = sym.Symbol('phi_j', real=True)

thetak = sym.Symbol('theta_k', real=True)
phik = sym.Symbol('phi_k', real=True)

sxj, syj, szj, jx, jy, jz, Wa, Wb, g, k, n = sym.symbols('s^x_j, s^y_j ,s^z_j, J_x, J_y, J_z, dW^a, dW^b, gamma, k, n', real=True)
sxk, syk, szk, dt = sym.symbols('s^x_k, s^y_k ,s^z_k, dt', real=True)

Ixk = sym.Sum(sxk, (k, 0, n))
Iyk = sym.Sum(syk, (k, 0, n))
Izk = sym.Sum(szk, (k, 0, n))

In [65]:
A = Matrix([[sym.cos(theta)*sym.cos(phi),-sym.sin(theta)*sym.sin(phi)],
        [-sym.cos(theta)*sym.sin(phi),-sym.sin(theta)*sym.cos(phi)],[sym.sin(theta),0]])
A

Matrix([
[ cos(phi_j)*cos(theta_j), -sin(phi_j)*sin(theta_j)],
[-sin(phi_j)*cos(theta_j), -sin(theta_j)*cos(phi_j)],
[            sin(theta_j),                        0]])

In [66]:
A_l = sym.simplify((sym.simplify((A.T@A).inv())@A.T))
A_l

Matrix([
[ cos(phi_j)*cos(theta_j), -sin(phi_j)*cos(theta_j), sin(theta_j)],
[-sin(phi_j)/sin(theta_j), -cos(phi_j)/sin(theta_j),            0]])

In [67]:
sym.simplify(A_l)

Matrix([
[ cos(phi_j)*cos(theta_j), -sin(phi_j)*cos(theta_j), sin(theta_j)],
[-sin(phi_j)/sin(theta_j), -cos(phi_j)/sin(theta_j),            0]])

In [70]:
dsx = (-g/2 * sxj + 2*jy *Iyk*szj - 2*jz*Izk*syj)*dt + (1+szj-sxj**2)*Wa*sym.sqrt(g/2) + sxj*syj*Wb*sym.sqrt(g/2)
dsy = (-g/2 * syj + 2*jz *Izk*sxj - 2*jx*szj*Ixk)*dt + (1+szj-syj**2)*Wb*sym.sqrt(g/2) + sxj*syj*Wa*sym.sqrt(g/2)
dsz = (-g * (1+ szj) + 2*jx *Ixk*syj - 2*jy*szj*Ixk)*dt + sxj*(1+szj)*Wa*g/2 + syj*(1+szj)*Wb*g/2

In [71]:
# dsx = (-g/2 * sxj + 2*jy *syk*szj - 2*jz*szk*syj)*dt + (1+szj-sxj**2)*Wa*sym.sqrt(g/2) + sxj*syj*Wb*sym.sqrt(g/2)
# dsy = (-g/2 * syj + 2*jz *szk*sxj - 2*jx*szj*sxk)*dt + (1+szj-syj**2)*Wb*sym.sqrt(g/2) + sxj*syj*Wa*sym.sqrt(g/2)
# dsz = (-g * (1+ szj) + 2*jx *sxk*syj - 2*jy*szj*sxk)*dt + sxj*(1+szj)*Wa*g/2 + syj*(1+szj)*Wb*g/2

In [72]:
dsx

sqrt(2)*dW^a*sqrt(gamma)*(-s^x_j**2 + s^z_j + 1)/2 + sqrt(2)*dW^b*sqrt(gamma)*s^x_j*s^y_j/2 + dt*(2*J_y*s^z_j*Sum(s^y_k, (k, 0, n)) - 2*J_z*s^y_j*Sum(s^z_k, (k, 0, n)) - gamma*s^x_j/2)

In [73]:
dsy

sqrt(2)*dW^a*sqrt(gamma)*s^x_j*s^y_j/2 + sqrt(2)*dW^b*sqrt(gamma)*(-s^y_j**2 + s^z_j + 1)/2 + dt*(-2*J_x*s^z_j*Sum(s^x_k, (k, 0, n)) + 2*J_z*s^x_j*Sum(s^z_k, (k, 0, n)) - gamma*s^y_j/2)

In [74]:
dsz

dW^a*gamma*s^x_j*(s^z_j + 1)/2 + dW^b*gamma*s^y_j*(s^z_j + 1)/2 + dt*(2*J_x*s^y_j*Sum(s^x_k, (k, 0, n)) - 2*J_y*s^z_j*Sum(s^x_k, (k, 0, n)) - gamma*(s^z_j + 1))

In [76]:
SXj = sym.sqrt(3)*sym.sin(theta)*sym.cos(phi)
SYj = sym.sqrt(3)*-sym.sin(theta)*sym.sin(phi)
SZj = sym.sqrt(3)*sym.cos(theta)

SXk = sym.Sum(sym.sqrt(3)*sym.sin(thetak)*sym.cos(phik), (k, 0, n))
SYk = sym.Sum(sym.sqrt(3)*-sym.sin(thetak)*sym.sin(phik), (k, 0, n))
SZk = sym.Sum(sym.sqrt(3)*sym.cos(thetak), (k, 0, n))


dsx_tp = dsx.subs({sxj:SXj, syj:SYj, szj:SZj, sxk:SXk, syk:SYk, szk:SZk})
dsy_tp = dsy.subs({sxj:SXj, syj:SYj, szj:SZj, sxk:SXk, syk:SYk, szk:SZk})
dsz_tp = dsz.subs({sxj:SXj, syj:SYj, szj:SZj, sxk:SXk, syk:SYk, szk:SZk})

In [77]:
# SXj = sym.sqrt(3)*sym.sin(theta)*sym.cos(phi)
# SYj = sym.sqrt(3)*-sym.sin(theta)*sym.sin(phi)
# SZj = sym.sqrt(3)*sym.cos(theta)

# SXk = sym.sqrt(3)*sym.sin(thetak)*sym.cos(phik)
# SYk = sym.sqrt(3)*-sym.sin(thetak)*sym.sin(phik)
# SZk = sym.sqrt(3)*sym.cos(thetak)


# dsx_tp = dsx.subs({sxj:SXj, syj:SYj, szj:SZj, sxk:SXk, syk:SYk, szk:SZk})
# dsy_tp = dsy.subs({sxj:SXj, syj:SYj, szj:SZj, sxk:SXk, syk:SYk, szk:SZk})
# dsz_tp = dsz.subs({sxj:SXj, syj:SYj, szj:SZj, sxk:SXk, syk:SYk, szk:SZk})

# dsx_tp = dsx.subs({sxj:SXj, syj:SYj, szj:SZj})
# dsy_tp = dsy.subs({sxj:SXj, syj:SYj, szj:SZj})
# dsz_tp = dsz.subs({sxj:SXj, syj:SYj, szj:SZj})

In [78]:
dsx_tp

sqrt(2)*dW^a*sqrt(gamma)*(-3*sin(theta_j)**2*cos(phi_j)**2 + sqrt(3)*cos(theta_j) + 1)/2 - 3*sqrt(2)*dW^b*sqrt(gamma)*sin(phi_j)*sin(theta_j)**2*cos(phi_j)/2 + dt*(2*sqrt(3)*J_y*cos(theta_j)*Sum(-sqrt(3)*sin(phi_k)*sin(theta_k), (k, 0, n), (k, 0, n)) + 2*sqrt(3)*J_z*sin(phi_j)*sin(theta_j)*Sum(sqrt(3)*cos(theta_k), (k, 0, n), (k, 0, n)) - sqrt(3)*gamma*sin(theta_j)*cos(phi_j)/2)

In [79]:
TP_eom = A_l@Matrix([dsx_tp, dsy_tp, dsz_tp])

In [80]:
TP_eom

Matrix([
[(sqrt(2)*dW^a*sqrt(gamma)*(-3*sin(theta_j)**2*cos(phi_j)**2 + sqrt(3)*cos(theta_j) + 1)/2 - 3*sqrt(2)*dW^b*sqrt(gamma)*sin(phi_j)*sin(theta_j)**2*cos(phi_j)/2 + dt*(2*sqrt(3)*J_y*cos(theta_j)*Sum(-sqrt(3)*sin(phi_k)*sin(theta_k), (k, 0, n), (k, 0, n)) + 2*sqrt(3)*J_z*sin(phi_j)*sin(theta_j)*Sum(sqrt(3)*cos(theta_k), (k, 0, n), (k, 0, n)) - sqrt(3)*gamma*sin(theta_j)*cos(phi_j)/2))*cos(phi_j)*cos(theta_j) - (-3*sqrt(2)*dW^a*sqrt(gamma)*sin(phi_j)*sin(theta_j)**2*cos(phi_j)/2 + sqrt(2)*dW^b*sqrt(gamma)*(-3*sin(phi_j)**2*sin(theta_j)**2 + sqrt(3)*cos(theta_j) + 1)/2 + dt*(-2*sqrt(3)*J_x*cos(theta_j)*Sum(sqrt(3)*sin(theta_k)*cos(phi_k), (k, 0, n), (k, 0, n)) + 2*sqrt(3)*J_z*sin(theta_j)*cos(phi_j)*Sum(sqrt(3)*cos(theta_k), (k, 0, n), (k, 0, n)) + sqrt(3)*gamma*sin(phi_j)*sin(theta_j)/2))*sin(phi_j)*cos(theta_j) + (sqrt(3)*dW^a*gamma*(sqrt(3)*cos(theta_j) + 1)*sin(theta_j)*cos(phi_j)/2 - sqrt(3)*dW^b*gamma*(sqrt(3)*cos(theta_j) + 1)*sin(phi_j)*sin(theta_j)/2 + dt*(-2*sqrt(3)*J_x*s

In [81]:
TP_eom[0]

(sqrt(2)*dW^a*sqrt(gamma)*(-3*sin(theta_j)**2*cos(phi_j)**2 + sqrt(3)*cos(theta_j) + 1)/2 - 3*sqrt(2)*dW^b*sqrt(gamma)*sin(phi_j)*sin(theta_j)**2*cos(phi_j)/2 + dt*(2*sqrt(3)*J_y*cos(theta_j)*Sum(-sqrt(3)*sin(phi_k)*sin(theta_k), (k, 0, n), (k, 0, n)) + 2*sqrt(3)*J_z*sin(phi_j)*sin(theta_j)*Sum(sqrt(3)*cos(theta_k), (k, 0, n), (k, 0, n)) - sqrt(3)*gamma*sin(theta_j)*cos(phi_j)/2))*cos(phi_j)*cos(theta_j) - (-3*sqrt(2)*dW^a*sqrt(gamma)*sin(phi_j)*sin(theta_j)**2*cos(phi_j)/2 + sqrt(2)*dW^b*sqrt(gamma)*(-3*sin(phi_j)**2*sin(theta_j)**2 + sqrt(3)*cos(theta_j) + 1)/2 + dt*(-2*sqrt(3)*J_x*cos(theta_j)*Sum(sqrt(3)*sin(theta_k)*cos(phi_k), (k, 0, n), (k, 0, n)) + 2*sqrt(3)*J_z*sin(theta_j)*cos(phi_j)*Sum(sqrt(3)*cos(theta_k), (k, 0, n), (k, 0, n)) + sqrt(3)*gamma*sin(phi_j)*sin(theta_j)/2))*sin(phi_j)*cos(theta_j) + (sqrt(3)*dW^a*gamma*(sqrt(3)*cos(theta_j) + 1)*sin(theta_j)*cos(phi_j)/2 - sqrt(3)*dW^b*gamma*(sqrt(3)*cos(theta_j) + 1)*sin(phi_j)*sin(theta_j)/2 + dt*(-2*sqrt(3)*J_x*sin(phi_j)*

In [82]:
TP_eom[1]

-(sqrt(2)*dW^a*sqrt(gamma)*(-3*sin(theta_j)**2*cos(phi_j)**2 + sqrt(3)*cos(theta_j) + 1)/2 - 3*sqrt(2)*dW^b*sqrt(gamma)*sin(phi_j)*sin(theta_j)**2*cos(phi_j)/2 + dt*(2*sqrt(3)*J_y*cos(theta_j)*Sum(-sqrt(3)*sin(phi_k)*sin(theta_k), (k, 0, n), (k, 0, n)) + 2*sqrt(3)*J_z*sin(phi_j)*sin(theta_j)*Sum(sqrt(3)*cos(theta_k), (k, 0, n), (k, 0, n)) - sqrt(3)*gamma*sin(theta_j)*cos(phi_j)/2))*sin(phi_j)/sin(theta_j) - (-3*sqrt(2)*dW^a*sqrt(gamma)*sin(phi_j)*sin(theta_j)**2*cos(phi_j)/2 + sqrt(2)*dW^b*sqrt(gamma)*(-3*sin(phi_j)**2*sin(theta_j)**2 + sqrt(3)*cos(theta_j) + 1)/2 + dt*(-2*sqrt(3)*J_x*cos(theta_j)*Sum(sqrt(3)*sin(theta_k)*cos(phi_k), (k, 0, n), (k, 0, n)) + 2*sqrt(3)*J_z*sin(theta_j)*cos(phi_j)*Sum(sqrt(3)*cos(theta_k), (k, 0, n), (k, 0, n)) + sqrt(3)*gamma*sin(phi_j)*sin(theta_j)/2))*cos(phi_j)/sin(theta_j)

In [83]:
sym.simplify(TP_eom[0])

3*sqrt(2)*dW^a*sqrt(gamma)*sin(phi_j)**2*sin(theta_j)**2*cos(phi_j)*cos(theta_j)/2 - 3*sqrt(2)*dW^a*sqrt(gamma)*sin(theta_j)**2*cos(phi_j)**3*cos(theta_j)/2 + sqrt(6)*dW^a*sqrt(gamma)*cos(phi_j)*cos(theta_j)**2/2 + sqrt(2)*dW^a*sqrt(gamma)*cos(phi_j)*cos(theta_j)/2 + 3*dW^a*gamma*sin(theta_j)**2*cos(phi_j)*cos(theta_j)/2 + sqrt(3)*dW^a*gamma*sin(theta_j)**2*cos(phi_j)/2 + 3*sqrt(2)*dW^b*sqrt(gamma)*sin(phi_j)**3*sin(theta_j)**2*cos(theta_j)/2 - 3*sqrt(2)*dW^b*sqrt(gamma)*sin(phi_j)*sin(theta_j)**2*cos(phi_j)**2*cos(theta_j)/2 - sqrt(6)*dW^b*sqrt(gamma)*sin(phi_j)*cos(theta_j)**2/2 - sqrt(2)*dW^b*sqrt(gamma)*sin(phi_j)*cos(theta_j)/2 - 3*dW^b*gamma*sin(phi_j)*sin(theta_j)**2*cos(theta_j)/2 - sqrt(3)*dW^b*gamma*sin(phi_j)*sin(theta_j)**2/2 - sqrt(3)*dt*gamma*sin(phi_j)**2*sin(theta_j)*cos(theta_j)/2 - sqrt(3)*dt*gamma*sin(theta_j)*cos(phi_j)**2*cos(theta_j)/2 - sqrt(3)*dt*gamma*sin(theta_j)*cos(theta_j) - dt*gamma*sin(theta_j) - 6*dt*(n + 1)**2*(J_x*sin(phi_j)*sin(theta_j)**2*cos(phi_k) 

In [84]:
sym.simplify(TP_eom[1])

3*sqrt(2)*dW^a*sqrt(gamma)*sin(phi_j)*sin(theta_j)*cos(phi_j)**2 - sqrt(6)*dW^a*sqrt(gamma)*sin(phi_j)/(2*tan(theta_j)) - sqrt(2)*dW^a*sqrt(gamma)*sin(phi_j)/(2*sin(theta_j)) + 3*sqrt(2)*dW^b*sqrt(gamma)*sin(phi_j)**2*sin(theta_j)*cos(phi_j) - sqrt(6)*dW^b*sqrt(gamma)*cos(phi_j)/(2*tan(theta_j)) - sqrt(2)*dW^b*sqrt(gamma)*cos(phi_j)/(2*sin(theta_j)) + 6*dt*(n + 1)**2*(J_x*sin(theta_k)*cos(phi_j)*cos(phi_k)/tan(theta_j) + J_y*sin(phi_j)*sin(phi_k)*sin(theta_k)/tan(theta_j) - J_z*(sin(phi_j)**2 + cos(phi_j)**2)*cos(theta_k))